<h1><center>Laboratorio 9:  ¿Superhéroe o Villano?  🦴</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Ricardo Espinoza
- Nombre de alumno 2: Cristóbal Guerra


### **Link de repositorio de GitHub:** `https://github.com/cristobalg68/Labs-Espinoza-Guerra.git`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [ ]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn import set_config

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif, chi2
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow.

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados.
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


## 1.0 Se importa la data

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# path_comics = '/content/drive/Othercomputers/Mi PC/Estudios/Universidad/Semestre_2023_1/Laboratorio_de_Programación_Científica_para_Ciencia_de_Datos/Laboratorios/Lab_9/df_comics.csv'
# path_comics_no_label = '/content/drive/Othercomputers/Mi PC/Estudios/Universidad/Semestre_2023_1/Laboratorio_de_Programación_Científica_para_Ciencia_de_Datos/Laboratorios/Lab_9/comics_no_label.csv'

path_comics = '/content/drive/My Drive/Lab_9/df_comics.csv'
path_comics_no_label = '/content/drive/My Drive/Lab_9/comics_no_label.csv'

In [ ]:
df_comics = pd.read_csv(path_comics)
df_comics_no_label = pd.read_csv(path_comics_no_label)
df_comics = df_comics.dropna(subset=['history_text'])
df_comics_no_label = df_comics_no_label.dropna(subset=['history_text'])

In [ ]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
df_comics_no_label.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,17,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,31,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
6,46,A.M.A.Z.O. (CW),NaN,NaN,22,The Anti Meta-human Adaptive Zootomic Organism...,"With its metallic body, it easily overpowered ...",90,100,100,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
class_id = {'Good': 0,
            'Bad': 1,
            'Neutral': 2
            }

df_comics['new_alignment'] = df_comics['alignment'].map(class_id)

## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset.

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen.

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?.

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [ ]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [ ]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [ ]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [ ]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,&,actor,beauti,best,citi,good,like,new,rock,roll,teacher,world,york
0,1,0,0,0,0,1,1,0,2,1,1,0,0
1,0,1,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados.

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [ ]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,&,& roll,actor,actor world,beauti,beauti citi,best,best actor,citi,good,...,rock,rock &,rock best,rock like,roll,teacher,teacher rock,world,york,york beauti
0,1,1,0,0,0,0,0,0,0,1,...,2,1,0,1,1,1,1,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,1,0,1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [X] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [X] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [ ]:
set_config(transform_output="pandas")

min_max = MinMaxScaler()
minmax_columns = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

bog = CountVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1,2))

preprocessor = ColumnTransformer(
    transformers=[('MinMaxScaler', min_max, minmax_columns),
                  ('CountVectorizer', bog, 'history_text')
                  ], sparse_threshold=0)

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`.

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [X] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [X] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [X] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [ ]:
Pipe1 = Pipeline([
            ('Se trasforman las columnas', preprocessor),
            ('MultinomialNB', MultinomialNB())
        ])

In [ ]:
Pipe2 = Pipeline([
            ('Se trasforman las columnas', preprocessor),
            ('DummyClassifier', DummyClassifier(strategy="stratified"))
        ])

In [ ]:
X = df_comics.drop(columns=['new_alignment', 'alignment'])
y = df_comics['new_alignment'].astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)
target_names = ['heroe', 'villano', 'neutral']

In [ ]:
Pipe1.fit(X_train, y_train)
y_pred_1 = Pipe1.predict(X_test)
print(classification_report(y_test, y_pred_1, target_names=target_names, zero_division=0))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

       heroe       0.62      0.98      0.76       234
     villano       0.71      0.10      0.17       122
     neutral       0.00      0.00      0.00        30

    accuracy                           0.63       386
   macro avg       0.44      0.36      0.31       386
weighted avg       0.60      0.63      0.52       386



Vemos que el modelo MultinomialNB tiene un buen rendimiento para predecir la clase "héroe", con una buena precisión (0.63) y alto recall (0.98). Sin embargo, tiene dificultades predecir las clases "villano" y "neutral", lo que se refleja en métricas de recall muy bajas (0.1 y 0 respectivamente), pero sí se nota que para la clase "villano" el modelo mantiene buena precisión (0.71), lo que indica que en general predice muy pocos ejemplos como "villano", pero cuando lo hace en general tiene razón. En términos de accuracy, el modelo obtiene 63%.

In [ ]:
Pipe2.fit(X_train, y_train)
y_pred_2 = Pipe2.predict(X_test)
print(classification_report(y_test, y_pred_2, target_names=target_names, zero_division=0))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

       heroe       0.61      0.60      0.61       234
     villano       0.39      0.42      0.40       122
     neutral       0.00      0.00      0.00        30

    accuracy                           0.49       386
   macro avg       0.33      0.34      0.34       386
weighted avg       0.50      0.49      0.49       386



Vemos que el modelo DummyClassifier tiene un rendimiento aceptable para predecir la clase "héroe", con una buena precisión (0.61) y recall (0.60). Sin embargo, también tiene dificultades predecir las clases "villano" y "neutral", lo que se refleja en bajas métricas de precisión (0.39 y 0 respectivamente) y recall (0.42 y 0 respectivamente), lo que indica que en general el modelo logra un equilibrio entre la cantidad de veces que intenta predecir un ejemplo como "héroe" o "villano", y la efectividad que tiene en términos de que la predicción sea correcta para cualquier ejemplo.

De los resultados anteriores, se nota que ambos modelos actúan de manera diferente. Por un lado, el modelo MultinomialNB es muy bueno prediciendo correctamente los ejemplos que pertenecen a la clase "héroe" (mucho mejor que el modelo DummyClassifier), pero con el costo de un mal rendimiento en la correcta predicción de las otras dos clases, donde se nota que el modelo en general parece no clasificar muchos ejemplos como "villano", pero cuando lo hace en general tiene razón. Por el otro lado, el modelo DummyClassifier obtiene un rendimiento más "equilibrado" a la hora de predecir correctamente cualquiera de las clases, notando que a diferencia del modelo MultinomialNB, este sí intenta clasificar más ejemplos como "villano" (y por ende tiene mejor recall), pero cuando lo hace en general no tiene tan buena precisión.

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [ ]:
params = [
    {
        'clf': SVC(random_state=22),
        'clf__C': [1, 10, 100],
        'clf__kernel': ['linear'],
        'clf__degree': [3, 5]
    },
    {
        'clf': RandomForestClassifier(random_state=22),
        'clf__n_estimators': [50, 100, 150],
        'clf__max_depth': [5, 10],
        'clf__criterion': ['gini', 'entropy']
    },
    {
        'clf': GradientBoostingClassifier(random_state=22),
        'clf__learning_rate': [0.1, 0.01],
        'clf__n_estimators': [100, 200],
        'clf__max_depth': [3, 5]
    }
]

CountVectorizer_params = {
    'preprocessor__CountVectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'preprocessor__CountVectorizer__max_df': [0.8, 0.9],
    'preprocessor__CountVectorizer__min_df': [0.5, 0.8]
}

selector_params = {
    'selector__percentile': [20, 40, 60, 80],
    'selector__score_func': [chi2]
}
BestParams = []

In [ ]:
Pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', SelectPercentile()),
    ('clf', params[0]['clf'])
])
params[0].pop('clf')
search = HalvingGridSearchCV(Pipe, {**CountVectorizer_params, **selector_params, **params[0]}, verbose=10)
search.fit(X_train, y_train)
BestParams.append(search.best_params_)

In [ ]:
Pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', SelectPercentile()),
    ('clf', params[1]['clf'])
])
params[1].pop('clf')
search = HalvingGridSearchCV(Pipe, {**CountVectorizer_params, **selector_params, **params[1]}, verbose=10)
search.fit(X_train, y_train)
BestParams.append(search.best_params_)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
Pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', SelectPercentile()),
    ('clf', params[2]['clf'])
])
params[2].pop('clf')
search = HalvingGridSearchCV(Pipe, {**CountVectorizer_params, **selector_params, **params[2]}, verbose=10)
search.fit(X_train, y_train)
BestParams.append(search.best_params_)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(BestParams)

[{'clf__criterion': 'entropy', 'clf__max_depth': 5, 'clf__n_estimators': 150, 'preprocessor__CountVectorizer__max_df': 0.9, 'preprocessor__CountVectorizer__min_df': 0.5, 'preprocessor__CountVectorizer__ngram_range': (1, 2), 'selector__percentile': 40, 'selector__score_func': <function chi2 at 0x7f023dd61b40>}, {'clf__learning_rate': 0.01, 'clf__max_depth': 3, 'clf__n_estimators': 100, 'preprocessor__CountVectorizer__max_df': 0.8, 'preprocessor__CountVectorizer__min_df': 0.5, 'preprocessor__CountVectorizer__ngram_range': (1, 2), 'selector__percentile': 40, 'selector__score_func': <function chi2 at 0x7f023dd61b40>}]


In [ ]:
set_config(transform_output="pandas")

min_max = MinMaxScaler()
minmax_columns = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

bog = CountVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1,2), max_df=0.9, min_df=0.5)

preprocessor = ColumnTransformer(
    transformers=[('MinMaxScaler', min_max, minmax_columns),
                  ('CountVectorizer', bog, 'history_text')
                  ], sparse_threshold=0)

Pipe3 = Pipeline([
        ('preprocessor', preprocessor),
        ('selector', SelectPercentile(percentile=20, score_func=chi2)),
        ('clf', SVC(random_state=22, C=10, kernel='linear', degree=3))
    ])

In [ ]:
Pipe3.fit(X_train, y_train)
y_pred_3 = Pipe3.predict(X_test)
print(classification_report(y_test, y_pred_3, target_names=target_names, zero_division=0))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

       heroe       0.61      1.00      0.75       234
     villano       0.00      0.00      0.00       122
     neutral       0.00      0.00      0.00        30

    accuracy                           0.61       386
   macro avg       0.20      0.33      0.25       386
weighted avg       0.37      0.61      0.46       386



In [ ]:
set_config(transform_output="pandas")

min_max = MinMaxScaler()
minmax_columns = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

bog = CountVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1,2), max_df=0.9, min_df=0.5)

preprocessor = ColumnTransformer(
    transformers=[('MinMaxScaler', min_max, minmax_columns),
                  ('CountVectorizer', bog, 'history_text')
                  ], sparse_threshold=0)

Pipe4 = Pipeline([
        ('preprocessor', preprocessor),
        ('selector', SelectPercentile(percentile=40, score_func=chi2)),
        ('clf', RandomForestClassifier(random_state=22, criterion='entropy', max_depth=5, n_estimators=150))
    ])

In [ ]:
Pipe4.fit(X_train, y_train)
y_pred_4 = Pipe4.predict(X_test)
print(classification_report(y_test, y_pred_4, target_names=target_names, zero_division=0))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

       heroe       0.62      0.96      0.75       234
     villano       0.50      0.10      0.16       122
     neutral       0.00      0.00      0.00        30

    accuracy                           0.61       386
   macro avg       0.37      0.35      0.31       386
weighted avg       0.53      0.61      0.51       386



In [ ]:
set_config(transform_output="pandas")

min_max = MinMaxScaler()
minmax_columns = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

bog = CountVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1,2), max_df=0.8, min_df=0.5)

preprocessor = ColumnTransformer(
    transformers=[('MinMaxScaler', min_max, minmax_columns),
                  ('CountVectorizer', bog, 'history_text')
                  ], sparse_threshold=0)

Pipe5 = Pipeline([
        ('preprocessor', preprocessor),
        ('selector', SelectPercentile(percentile=40, score_func=chi2)),
        ('clf', GradientBoostingClassifier(random_state=22, learning_rate=0.01, max_depth=3, n_estimators=100))
    ])

In [ ]:
Pipe5.fit(X_train, y_train)
y_pred_5 = Pipe5.predict(X_test)
print(classification_report(y_test, y_pred_5, target_names=target_names, zero_division=0))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

       heroe       0.62      0.97      0.76       234
     villano       0.59      0.08      0.14       122
     neutral       0.00      0.00      0.00        30

    accuracy                           0.62       386
   macro avg       0.40      0.35      0.30       386
weighted avg       0.56      0.62      0.50       386



Se reportan nuevamente los resultados de los modelos.

SVC

                  precision    recall  f1-score   support

          heroe       0.61      1.00      0.75       234
        villano       0.00      0.00      0.00       122
        neutral       0.00      0.00      0.00        30

        accuracy                          0.61       386
       macro avg      0.20      0.33      0.25       386
    weighted avg      0.37      0.61      0.46       386

RandomForest

                  precision    recall  f1-score   support

          heroe       0.62      0.96      0.75       234
        villano       0.50      0.10      0.16       122
        neutral       0.00      0.00      0.00        30

        accuracy                          0.61       386
       macro avg      0.37      0.35      0.31       386
    weighted avg      0.53      0.61      0.51       386

GradientBoosting

                  precision    recall  f1-score   support

          heroe       0.62      0.97      0.76       234
        villano       0.59      0.08      0.14       122
        neutral       0.00      0.00      0.00        30

        accuracy                          0.62       386
       macro avg      0.40      0.35      0.30       386
    weighted avg      0.56      0.62      0.50       386

De los anteriores resultados, se concluye que el mejor modelo es el de Random Forest, ya que logró una precisión del 62% para la clase "heroe", i.e. el 62% de las predicciones para esta clase fueron correctas. Además, su recall para la clase "héroe" fue del 96%, esto indica que el modelo identificó correctamente al 96% de los ejemplos positivos reales de esta clase. El valor F1-score de 0.75 indica un equilibrio entre precision y recall. Para las clases "villano" y "neutral", si bien los resultados son más bajos comparativamente a aquellos para la clase "héroe", el modelo de Random Forest también superó a los otros clasificadores en términos de rendimiento general, con una precisión y recall del 50% y 10% respectivamente en la clase "villano", además de un F1-score de 0.16 que es superior a aquel logrado por el modelo de GradientBoosting.

En particular, para el modelo de RF se selecciona el 40% de las características más importantes según el score_func de chi2, ya que esto ayuda a mejorar la relevancia de las características y reduce la dimensionalidad. También se utiliza el criterio 'entropy' para la medida de impureza en los nodos del árbol, max_depth = 5 para controlar la profundidad máxima del árbol y evitar un posible sobreajuste.

Con respecto a los parámetros de CountVectorizer seleccionados por GridSearch, estos son:
* Tokenizer: StemmerTokenizer()
* n-gram: (1,2), es decir se consideran palabras individuales y combinaciones de dos palabras seguidas.
* max_df: 0.9, es decir se eliminan las palabras que aparecen en más del 90% de los datos para quitar ruido y términos que no aportan información.
* min_df: 0.5, es decir se eliminan las palabras que aparecen en menos del 50% de los datos para filtrar palabras poco comunes que pueden ser ruido.

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [ ]:
personajes = ['Vergil', 'Gorilla Girl', 'Batcow']
df_comics_no_label_personajes = df_comics_no_label[df_comics_no_label['name'].isin(personajes)].copy()
df_comics_no_label_personajes.drop_duplicates(subset=['name'], inplace=True)
df_comics_no_label_personajes

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
16,122,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,529,Gorilla Girl,Fahnbullah Eddy,Fahnbullah Eddy,7,A carnival performer with the ability to turn ...,Gorilla Girl can transform into a talking gori...,90,35,60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
78,1368,Vergil,Vergil Sparda,NaN,16,"Vergil, later also known as Nelo Angelo, is on...",NaN,90,75,95,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
y_pred_4_no_label = Pipe4.predict(df_comics_no_label_personajes)
y_pred_4_no_label

array([0, 0, 0])

Como se puede ver aquí, el modelo predice que los tres personajes son héroes.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>